In [1]:
import pandas as pd

In [2]:
users = pd.read_csv('sushi3.udata', sep='\t', names=('uid', 'gender', 'age', 'time', 'old_prefecture', 'old_region', 'old_eastwest', 'prefecture', 'region', 'eastwest', 'same'))

In [3]:
users.head()

,uid,gender,age,time,old_prefecture,old_region,old_eastwest,prefecture,region,eastwest,same
0,6371,0,2,355,0,0,0,0,0,0,0
1,10007,1,1,214,26,6,1,26,6,1,0
2,1777,1,2,363,29,6,1,26,6,1,1
3,3613,0,4,395,40,9,1,26,6,1,1
4,8081,1,1,707,26,6,1,36,8,1,1


In [4]:
users['time'].max()

877

In [5]:
items = pd.read_csv('sushi3.idata', sep='\t', names=('iid', 'name', 'style', 'major', 'minor', 'heaviness', 'frequency', 'price', 'popularity'))

In [6]:
R = pd.read_csv('sushi3b.5000.10.score', sep=' ', header=None)

In [7]:
import numpy as np

In [8]:
triplets = []
for i, line in enumerate(np.array(R)):
    for j, v in enumerate(line):
        if v != -1:
            triplets.append((i, j, v))

In [9]:
triplets = pd.DataFrame(triplets)

In [10]:
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

In [11]:
len(triplets)

50000

In [12]:
reader = Reader(rating_scale=(0, 4))
data = Dataset.load_from_df(triplets, reader)

In [13]:
cross_validate(NormalPredictor(), data, cv=5)

{'test_rmse': array([1.67728431, 1.66162576, 1.64635649, 1.66936441, 1.64984544]),
 'test_mae': array([1.33548377, 1.32305269, 1.31240066, 1.33518886, 1.30948637]),
 'fit_time': (0.04904794692993164,
  0.059880971908569336,
  0.06375789642333984,
  0.05938267707824707,
  0.059957027435302734),
 'test_time': (0.1363687515258789,
  0.06699490547180176,
  0.10964012145996094,
  0.06799674034118652,
  0.11223101615905762)}

In [14]:
from surprise import BaselineOnly

cross_validate(BaselineOnly(), data, cv=5)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([1.1453357 , 1.14573844, 1.15469846, 1.14166872, 1.13637492]),
 'test_mae': array([0.93761212, 0.93457427, 0.94300041, 0.93410233, 0.92740529]),
 'fit_time': (0.11309432983398438,
  0.13320255279541016,
  0.11574029922485352,
  0.12347936630249023,
  0.11876320838928223),
 'test_time': (0.051605939865112305,
  0.09645247459411621,
  0.052013397216796875,
  0.08857560157775879,
  0.05341339111328125)}

In [15]:
from surprise import SVD

In [16]:
cross_validate(SVD(n_factors=10, n_epochs=20), data, cv=5)

{'test_rmse': array([1.14201016, 1.14097713, 1.12220179, 1.13729438, 1.1393489 ]),
 'test_mae': array([0.92381361, 0.92429428, 0.90822234, 0.92654559, 0.92607855]),
 'fit_time': (0.8296024799346924,
  0.8453769683837891,
  0.8499362468719482,
  0.8429224491119385,
  0.849799394607544),
 'test_time': (0.1134042739868164,
  0.0767831802368164,
  0.11213040351867676,
  0.07719135284423828,
  0.1117098331451416)}

In [17]:
u_cont = {'time'}
u_cat = set(users.columns) - u_cont
u_cat

{'age',
 'eastwest',
 'gender',
 'old_eastwest',
 'old_prefecture',
 'old_region',
 'prefecture',
 'region',
 'same',
 'uid'}

In [18]:
i_cont = {'heaviness', 'frequency', 'price', 'popularity'}
i_cat = set(items.columns) - i_cont - {'name'}
i_cat

{'iid', 'major', 'minor', 'style'}

In [19]:
Xi = []
Xv = []
y = []
for i, j, v in np.array(triplets):
    indexes = []
    values = []
    
    indexes.extend(list(users.iloc[i][u_cat]))
    values.extend([1] * len(u_cat))
    indexes.extend([1] * len(u_cont))
    values.extend(list(users.iloc[i][u_cont] / 877))
    
    indexes.extend(list(items.iloc[j][i_cat]))
    values.extend([1] * len(i_cat))
    indexes.extend([1] * len(i_cont))
    values.extend(list(items.iloc[j][i_cont]))
    
    assert len(indexes) == len(values)
    Xi.append(indexes)
    Xv.append(values)
    y.append(v)

In [20]:
len(Xi), len(Xv), len(y), len(Xi[0])

(50000, 50000, 50000, 19)

In [21]:
import pickle

with open('sushi.pickle', 'wb') as f:
    # pickle.dump(Xi, f, pickle.HIGHEST_PROTOCOL)
    # pickle.dump(Xv, f, pickle.HIGHEST_PROTOCOL)
    # pickle.dump(y, f, pickle.HIGHEST_PROTOCOL)
    pickle.dump(Xi, f, 2)
    pickle.dump(Xv, f, 2)
    pickle.dump(y, f, 2)

In [22]:
users.shape

(5000, 11)

In [23]:
items.shape

(100, 9)